<a href="https://colab.research.google.com/github/mucahitozkaya/app/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import keras
from keras import layers
from tensorflow.keras.utils import to_categorical


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Activation,Dropout, BatchNormalization,MaxPool1D,LayerNormalization
from tensorflow.keras import Sequential
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC  # Import Support Vector Classifier (SVC)
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Input
from logging import Logger
from tensorflow.keras.models import Model
from keras.regularizers import l1,l2

#Read datas

In [2]:
train_final = pd.read_parquet("https://drive.google.com/uc?id=1ZeXb2EBMkP-nzULonq_hDnIxF5u9PwKp")
test_final = pd.read_parquet("https://drive.google.com/uc?id=1Blepx7rAAUXFLLNIL8gCI2g1IcNoMfTk")
submission_sample = pd.read_parquet("https://drive.google.com/uc?id=1KCcrx84tmG5biO0MjLmnFhvCMSC-U6ge")

In [3]:
dfx = train_final.copy()
df_testx = test_final.copy()

In [ ]:
# Alt alta grafikleri çiz
fig, axes = plt.subplots(nrows=1, figsize=(8, 5 ))

ax = sns.countplot(x='devicebrand', hue=dfx['target1'], data=dfx, ax = axes)


In [ ]:
a = dfx['carrier'].value_counts()
a
for i in a:
  print(i)
a

In [ ]:
# 'devicecbrand' sütunundaki değerlerin sayısını alın
value_counts = dfx['devicebrand'].value_counts()

selected_values = value_counts[value_counts >= 200].index.tolist()

dfx = dfx[dfx['devicebrand'].isin(selected_values)]

In [ ]:
value_counts = dfx['carrier'].value_counts()

selected_values = value_counts[value_counts >= 100].index.tolist()

dfx = dfx[dfx['carrier'].isin(selected_values)]

In [ ]:
# devicecbrand sütununun value_counts'ını alalım
value_counts = dfx['devicebrand'].value_counts()

# 200'den az olan değerlere 'OTHER' yazalım
threshold = 200
dfx['devicebrand'] = dfx['devicebrand'].apply(lambda x: 'OTHER' if value_counts[x] < threshold else x)


In [ ]:
# devicecbrand sütununun value_counts'ını alalım
value_counts = dfx['carrier'].value_counts()

# 200'den az olan değerlere 'OTHER' yazalım
threshold = 100
dfx['carrier'] = dfx['carrier'].apply(lambda x: 'OTHER' if value_counts[x] < threshold else x)

In [ ]:
dfx['carrier'].nunique()

12

In [ ]:
label_encoder = LabelEncoder()
dfx['target_encoded'] = label_encoder.fit_transform(dfx['target'])
dfx['target_encoded'].nunique()

112

In [ ]:
features_corr = df.loc[:, 'feature_0':'feature_24']
features_corr['target_encoded'] = df['target_encoded']
corr_matrix = features_corr.corr()

plt.figure(figsize=(24, 18))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title("Sütunlar Arasındaki Korelasyon")
plt.show()

In [ ]:
features_corr = df.loc[:, 'feature_25':'target_encoded']
features_corr = features_corr.drop(columns=['target'])
corr_matrix = features_corr.corr()

plt.figure(figsize=(24, 18))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title("Sütunlar Arasındaki Korelasyon")
plt.show()

In [4]:
dfx['target_cleaned'] = dfx['target'].apply(lambda x: ','.join([value.strip() for value in x.split(',')]))

dfx[['target1', 'target2','target3']] = dfx['target_cleaned'].str.split(',', expand=True)

dfx.drop(columns=['target','target_cleaned'], inplace=True)
dfx

,id,month,n_seconds_1,n_seconds_2,n_seconds_3,carrier,devicebrand,feature_0,feature_1,feature_2,...,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target1,target2,target3
0,5beefd4d2bf4a4767e0df8108,10,5245.571,981.182,205.948,VODAFONE TR,Apple,-1.197737,1.113360,-1.123334,...,2.645719,-1.023478,1.658986,-1.559406,-2.161336,30,58,menu2,menu4,menu5
1,867285b116c063d5a8482f5be,10,5184.876,557.650,487.587,TURKCELL,samsung,-2.336352,2.567766,-0.494908,...,-0.021547,-0.195770,2.775513,-0.318980,-4.291473,21,45,menu7,menu8,menu4
2,c82a7cbd2e00d9b66c06bcadc,10,3835.618,3275.128,43.806,TURK TELEKOM,Redmi,-2.561455,2.061736,-0.184511,...,1.673868,0.631790,1.293131,-2.230909,-2.383524,19,61,menu2,menu8,menu4
3,f2d2b25073ccc298eced86897,10,3532.544,154.509,64.724,TURKCELL,samsung,-2.529918,3.358050,-0.851366,...,1.863495,0.213170,1.029710,-1.142185,-4.466191,2,41,menu6,menu2,menu1
4,7818c92a58af0f2cb7c361738,10,3344.192,787.896,715.115,VODAFONE TR,samsung,-2.922361,2.096124,0.060796,...,1.024499,-0.186423,-0.061626,-1.462175,-2.371206,23,85,menu6,menu2,menu8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94044,2e54f32ced9fae6ef802ceaa2,12,44.397,43.425,41.678,O2 - DE,HUAWEI,-1.531534,2.596604,0.340233,...,0.044385,0.863442,0.761354,1.073711,-3.560019,29,68,menu2,menu8,menu4
94045,6aa4ff0f6cc5ef4c2980b2862,12,44.331,43.977,40.620,TURKCELL,samsung,-1.268987,2.300487,0.231711,...,2.406462,-0.340132,-0.893553,-2.061401,-1.908158,25,43,menu9,menu2,menu5
94046,fa842185a0edd210845b78308,12,44.142,43.591,41.736,VODAFONE TR,Apple,-1.950039,2.805681,0.438200,...,0.449427,0.445361,-0.495204,-0.955097,-3.843092,4,25,menu6,menu2,menu4
94047,db8f55b8499f8d8c05148240e,12,43.963,43.350,40.862,TURK TELEKOM,Redmi,-2.389140,2.358281,0.683524,...,0.864857,1.284815,-0.100755,-1.056479,-1.452037,25,60,menu6,menu8,menu4


In [ ]:
dfx['target1'].value_counts()


menu6    58581
menu9    15654
menu2    13703
menu3     2922
menu7     1130
menu8      362
menu1      106
menu4      103
menu5       36
Name: target1, dtype: int64

In [ ]:
df['target1'].unique()

array(['menu2', 'menu7', 'menu6', 'menu9', 'menu3', 'menu8', 'menu1',
       'menu4', 'menu5'], dtype=object)

In [ ]:
df['id'].nunique()
df.drop(columns=['id'], inplace=True)

In [ ]:
df.info()


#Call Back

In [5]:
from tensorflow.keras.callbacks import ModelCheckpoint
import datetime

# eğitim sırasında en iyi model ağırlıklarını kayıt edelim ve
# eğitim performansını TensorBoard ile izleyelim

filepath="weights-improvement-{epoch:02d}-{val_output3_sparse_categorical_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_output3_sparse_categorical_accuracy', verbose=1, save_best_only=True, mode='max')
logdir="/content/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
callbacks_list = [checkpoint,tensorboard_callback]


#Preprocess

In [6]:
df = dfx.copy()
df_test = df_testx

In [7]:


menu_etiketleri = {'menu1': 0, 'menu2': 1, 'menu3': 2, 'menu4': 3,'menu5': 4, 'menu6': 5, 'menu7': 6, 'menu8': 7, 'menu9':8}

df['target1'] = df['target1'].map(menu_etiketleri)
df['target2'] = df['target2'].map(menu_etiketleri)
df['target3'] = df['target3'].map(menu_etiketleri)

In [ ]:
df['target3'].value_counts()

2    26029
6    19460
3    18868
0    16538
7     5905
4     4236
1     2836
5      177
Name: target3, dtype: int64

In [ ]:
label_encoder = LabelEncoder()
df['target1'] = label_encoder.fit_transform(df['target1'])
df['target2'] = label_encoder.fit_transform(df['target2'])
df['target3'] = label_encoder.fit_transform(df['target3'])

In [8]:
X = df.drop(["target1", "target2", "target3"], axis=1)
y1 = df[["target1"]]
y2 = df[["target2"]]
y3 = df[["target3"]]

In [ ]:
df['target2'].value_counts()


1    57833
3    12684
6     9429
0     5704
5     2809
2     2653
7     1615
4     1322
Name: target2, dtype: int64

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90254 entries, 0 to 94048
Data columns (total 57 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           90254 non-null  object 
 1   month        90254 non-null  int64  
 2   n_seconds_1  90254 non-null  float64
 3   n_seconds_2  90254 non-null  float64
 4   n_seconds_3  90254 non-null  float64
 5   carrier      90254 non-null  object 
 6   devicebrand  90254 non-null  object 
 7   feature_0    90254 non-null  float64
 8   feature_1    90254 non-null  float64
 9   feature_2    90254 non-null  float64
 10  feature_3    90254 non-null  float64
 11  feature_4    90254 non-null  float64
 12  feature_5    90254 non-null  float64
 13  feature_6    90254 non-null  float64
 14  feature_7    90254 non-null  float64
 15  feature_8    90254 non-null  float64
 16  feature_9    90254 non-null  float64
 17  feature_10   90254 non-null  float64
 18  feature_11   90254 non-null  float64
 19  feat

In [9]:
combined_data = pd.concat([X, df_test], axis=0)


# devicecbrand sütununun value_counts'ını alalım
value_counts = combined_data['devicebrand'].value_counts()

# 200'den az olan değerlere 'OTHER' yazalım
threshold = 200
combined_data['devicebrand'] = combined_data['devicebrand'].apply(lambda x: 'OTHER' if value_counts[x] < threshold else x)


# devicecbrand sütununun value_counts'ını alalım
value_counts = combined_data['carrier'].value_counts()

# 200'den az olan değerlere 'OTHER' yazalım
threshold = 100
combined_data['carrier'] = combined_data['carrier'].apply(lambda x: 'OTHER' if value_counts[x] < threshold else x)

In [ ]:
combined_data['devicebrand'].unique()

array(['Apple', 'samsung', 'Redmi', 'HUAWEI', 'POCO', 'OPPO', 'Sony',
       'vivo', 'OTHER', 'gm', 'xiaomi', 'reeder', 'HONOR', 'Lenovo',
       'realme', 'GM', 'Vestel', 'TECNO', 'lge', 'Casper', 'OnePlus'],
      dtype=object)

In [10]:

# combined_data = pd.concat([X, df_test], axis=0)

# combined_data.drop(columns=['id'], inplace=True)
label_encoder = LabelEncoder()
combined_data['id'] = label_encoder.fit_transform(combined_data['id'])
combined_data['carrier'] = label_encoder.fit_transform(combined_data['carrier'])
combined_data['devicebrand'] = label_encoder.fit_transform(combined_data['devicebrand'])

# Katogorik verilerin one hot encoding yapılarak numerik hale getirilmesi
object_list = []
for column in combined_data.columns:
     if combined_data[column].dtype == 'object':
        object_list.append(column)
# try:
#   object_list.remove('id')
#   df = df.drop(columns=['id'])
# except:
#   pass

combined_data = pd.get_dummies(combined_data, columns=object_list)

scaler = StandardScaler()
columns = combined_data.columns
combined_data = scaler.fit_transform(combined_data)

combined_data = pd.DataFrame(combined_data, columns = columns)

# pca = PCA(n_components=300)
# combined_data = pca.fit_transform(combined_data)

X = combined_data[:len(X)]
df_test = combined_data[len(X):]

In [11]:
X

,id,month,n_seconds_1,n_seconds_2,n_seconds_3,carrier,devicebrand,feature_0,feature_1,feature_2,...,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,-0.489540,-0.044583,11.501847,4.726212,1.318138,1.147539,-1.122579,0.884680,-1.137142,-1.307561,...,-0.179836,-1.789975,-0.206217,1.748168,-1.625553,1.132880,-1.021626,0.267382,1.133477,0.540392
1,0.083433,-0.044583,11.357055,2.180987,4.865938,-0.069142,1.080485,-0.729109,0.822148,-0.568963,...,-0.334394,0.339849,0.510566,-1.186294,-0.582019,2.471284,0.419019,-3.052173,0.053793,-0.458352
2,0.972470,-0.044583,8.138316,18.511737,-0.724361,-0.474702,0.101345,-1.048154,0.140454,-0.204150,...,-0.010087,-0.555222,-2.145001,0.678961,0.461329,0.694322,-1.801517,-0.078871,-0.186137,0.770872
3,1.553123,-0.044583,7.415314,-0.241699,-0.460858,-0.069142,1.080485,-1.003456,1.886772,-0.987913,...,-1.157101,-0.807126,0.800297,0.887583,-0.066448,0.378554,-0.537061,-3.324449,-2.225540,-0.765658
4,-0.108136,-0.044583,6.965989,3.564655,7.732104,1.147539,1.080485,-1.559676,0.186780,0.084162,...,1.511667,1.439186,-0.011462,-0.035459,-0.570235,-0.929653,-0.908701,-0.059675,0.293723,2.614708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94044,-1.101740,0.555313,-0.905877,-0.909261,-0.751168,-2.096943,-0.633010,0.411581,0.860997,0.412589,...,0.262235,-0.313995,-0.150873,-1.113758,0.753385,0.056870,2.036506,-1.912292,1.013512,1.308657
94045,-0.290173,0.555313,-0.906035,-0.905944,-0.764495,-0.069142,1.080485,0.783695,0.462085,0.285041,...,0.003959,0.798865,0.299114,1.484943,-0.764024,-1.926901,-1.604648,0.661929,0.533653,-0.612005
94046,1.656193,0.555313,-0.906486,-0.908264,-0.750437,1.147539,-1.122579,-0.181578,1.142654,0.527731,...,1.791076,1.867646,-1.993926,-0.668140,0.226288,-1.449392,-0.319775,-2.353426,-1.985610,-1.994882
94047,1.241706,0.555313,-0.906913,-0.909712,-0.761447,-0.474702,0.101345,-0.803928,0.539943,0.816063,...,1.227178,1.382772,-0.573063,-0.211094,1.284632,-0.976557,-0.437520,1.372737,0.533653,0.694045


In [ ]:
scaler = StandardScaler()
columns = df_test.columns
df_test = scaler.fit_transform(df_test)

df_test = pd.DataFrame(df_test, columns = columns)

# pca = PCA(n_components=30)
# df_test = pca.fit_transform(df_test)



In [25]:
# X = df.drop(["target1", "target2", "target3"], axis=1)
# y1 = df[["target1"]]
# y2 = df[["target2"]]
# y3 = df[["target3"]]
# scaler = StandardScaler()
# columns = X.columns
# X = scaler.fit_transform(X)
# X = pd.DataFrame(X, columns = columns)
# pca = PCA(n_components=30)
# X = pca.fit_transform(X)


# Veriyi eğitim ve test setlerine bölelim
X_train, X_test, y1_train, y1_test, y2_train, y2_test, y3_train, y3_test = train_test_split(
    X, y1, y2, y3, test_size=0.2, random_state=42
)

# Modeli oluşturma
input_layer = Input(shape=(57,))
embedding_layer = Embedding(input_dim=5000, output_dim=64)(input_layer)
lstm_layer = LSTM(64)(embedding_layer)

dropout_layer = Dropout(0.4)(lstm_layer)


l1_reg = 0.01
regularized_layer = Dense(40, activation='relu', kernel_regularizer=l1(l1_reg))(dropout_layer)

# İlk çıkış
output1 = Dense(20, activation='softmax', name='output1')(regularized_layer)

# İkinci çıkış
output2 = Dense(20, activation='softmax', name='output2')(regularized_layer)

# Üçüncü çıkış
output3 = Dense(20, activation='softmax', name='output3')(regularized_layer)

# Modeli oluştur
model = Model(inputs=input_layer, outputs=[output1, output2, output3])

# Modeli derle
model.compile(optimizer='adam',
              loss={'output1': 'sparse_categorical_crossentropy',
                    'output2': 'sparse_categorical_crossentropy',
                    'output3': 'sparse_categorical_crossentropy'},
              metrics={'output1': 'sparse_categorical_accuracy',
                       'output2': 'sparse_categorical_accuracy',
                       'output3': 'sparse_categorical_accuracy'})

# Modeli eğit
model.fit(X_train, {'output1': y1_train, 'output2': y2_train, 'output3': y3_train},
          validation_data=(X_test, {'output1': y1_test, 'output2': y2_test, 'output3': y3_test}),
          epochs=100, batch_size=32,callbacks=callbacks_list)

# Modeli değerlendir
loss_and_metrics = model.evaluate(X_test, [y1_test, y2_test, y3_test])
loss1 = loss_and_metrics[0]
accuracy1 = loss_and_metrics[1]
loss2 = loss_and_metrics[2]
accuracy2 = loss_and_metrics[3]
loss3 = loss_and_metrics[4]
accuracy3 = loss_and_metrics[5]

Epoch 1/100
2348/2352 [============================>.] - ETA: 0s - loss: 4.5895 - output1_loss: 1.1024 - output2_loss: 1.3339 - output3_loss: 1.7625 - output1_sparse_categorical_accuracy: 0.6318 - output2_sparse_categorical_accuracy: 0.6095 - output3_sparse_categorical_accuracy: 0.2721
Epoch 1: val_output3_sparse_categorical_accuracy did not improve from 0.37698
2352/2352 [==============================] - 28s 10ms/step - loss: 4.5891 - output1_loss: 1.1023 - output2_loss: 1.3340 - output3_loss: 1.7625 - output1_sparse_categorical_accuracy: 0.6318 - output2_sparse_categorical_accuracy: 0.6095 - output3_sparse_categorical_accuracy: 0.2720 - val_loss: 4.1430 - val_output1_loss: 1.0822 - val_output2_loss: 1.2972 - val_output3_loss: 1.7412 - val_output1_sparse_categorical_accuracy: 0.6279 - val_output2_sparse_categorical_accuracy: 0.6194 - val_output3_sparse_categorical_accuracy: 0.2757
Epoch 2/100
2346/2352 [============================>.] - ETA: 0s - loss: 4.1405 - output1_loss: 1.0776 -

In [13]:

model2 =  tf.keras.models.load_model("/content/weights-improvement-57-0.3770.hdf5")

# Tahminleri alın
predictions = model2.predict(df_test)

# Her bir çıkış için tahminleri ayırın
predicted_output1 = predictions[0]
predicted_output2 = predictions[1]
predicted_output3 = predictions[2]




374/374 [==============================] - 1s 3ms/step


In [14]:
print(predicted_output1[67])
print(predicted_output2[67])
print(predicted_output3[67])

[4.9152580e-04 2.6400274e-01 2.9449541e-02 4.0638825e-04 1.9123385e-04
 5.7769692e-01 9.9459030e-03 5.5646310e-03 1.1225110e-01]
[3.9487369e-02 6.2013954e-01 1.6550228e-02 1.0482431e-01 2.5921107e-10
 3.4672441e-03 9.1631627e-03 2.0152375e-01 4.8444243e-03]
[1.25715092e-01 1.71405040e-02 3.01716874e-10 2.06797913e-01
 1.13687836e-01 1.88282523e-02 2.93602562e-03 4.87578452e-01
 2.73159426e-02]


In [17]:
predicted_out1 = np.argmax(predicted_output1[60])
predicted_out2 = np.argmax(predicted_output2[60])
predicted_out3 = np.argmax(predicted_output3[60])


print(predicted_out1)
print(predicted_out2)
print(predicted_out3)

5
1
4


In [16]:

if predicted_out2 == predicted_out1:
  predicted_out2 = np.argsort(predicted_output2[67])[-2]


if predicted_out3 == predicted_out1 or predicted_out3 == predicted_out2:
  predicted_out3 = np.argsort(predicted_output3[67])[-2]


if predicted_out3 == predicted_out1 or predicted_out3 == predicted_out2:
  predicted_out3 = np.argsort(predicted_output3[67])[-3]


print(predicted_out1)
print(predicted_out2)
print(predicted_out3)

5
1
7


In [18]:
for i in range(len(predicted_output1)):
  predicted_out1 = np.argmax(predicted_output1[i])
  predicted_out2 = np.argmax(predicted_output2[i])
  predicted_out3 = np.argmax(predicted_output3[i])

  if predicted_out2 == predicted_out1:
    predicted_out2 = np.argsort(predicted_output2[67])[-2]


  if predicted_out3 == predicted_out1 or predicted_out3 == predicted_out2:
    predicted_out3 = np.argsort(predicted_output3[67])[-2]


  if predicted_out3 == predicted_out1 or predicted_out3 == predicted_out2:
    predicted_out3 = np.argsort(predicted_output3[67])[-3]

  binary_array = [0] * 9

  binary_array[predicted_out1] = 1
  binary_array[predicted_out2] = 1
  binary_array[predicted_out3] = 1

  binary_string = ''.join(map(str, binary_array))
  submission_sample.at[i, 'target'] = binary_string


In [19]:
submission_sample

,id,target
0,2e6105f5911256f4f6c4813ed,010101000
1,c56ad71dae0a5dbd3e7d36adc,010101000
2,4d02ea175f6581f0c6385311f,010001010
3,3412d27a86c286ba078fa935c,010001010
4,0203b561f6f7e10eafa46eefa,010001010
...,...,...
11950,7687113f46112edf4f56666ee,010101000
11951,5ff8eb7a06fd48b60dbc04f34,010101000
11952,ac23a7b9ad3e5d61e738c854b,010011000
11953,7da05018634ea2eee4b122756,010101000


In [20]:
# Define the file path for the CSV file
csv_file_path = 'submission_sample6.csv'

# Export the DataFrame to CSV
submission_sample.to_csv(csv_file_path, index=False)


In [22]:
dff = pd.read_csv("/content/submission_sample6.csv")
dff = dff.astype(str)


def count_non_three_ones(df, column_name='target'):
    count = 0
    for target in df[column_name]:
        if target.count('1') != 3:
            count += 1
            print(target)
    return count


count_non_three_ones(dff)

0

In [ ]:
def build_lstm_model(neurons, input_shape, lr, num_classes, activation, loss):
    try:
        #Logger.info("Building LSTM Model...")
        opt = tf.keras.optimizers.Adam(learning_rate=lr)
        inputs = tf.keras.layers.Input(shape=input_shape)
        x = tf.keras.layers.Embedding(input_dim=5000, output_dim=10)(inputs)
        x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(neurons))(x)
        # Define separate output layers for each target

        output_1 = tf.keras.layers.Dense(num_classes, activation=activation, name="output_1")(x)
        output_2 = tf.keras.layers.Dense(num_classes, activation=activation, name="output_2")(x)
        output_3 = tf.keras.layers.Dense(num_classes, activation=activation, name="output_3")(x)
        # Define the model with multiple outputs
        model = tf.keras.Model(inputs=inputs, outputs=[ output_1, output_2, output_3])
        model.compile(loss=loss, optimizer=opt, metrics=["sparse_categorical_accuracy"])
        #Logger.debug("LSTM Model Built...")
        return model
    except Exception as e:
        print("Exception while building model:", e)
        raise


X = df.drop(["target1", "target2", "target3"], axis=1)
y = df[["target1", "target2", "target3"]]

y = np.array(y)

(X, x_test, y, y_test) = train_test_split(X, y, random_state=100, test_size=0.1, shuffle=True)


(x_train, x_val, y_train, y_val) = train_test_split(X, y, random_state=100, test_size=0.1, shuffle=True)

#Variables
neurons = 100
input_shape = 55
lr = 0.001
activation = 'softmax'
loss = 'sparse_categorical_crossentropy'
num_classes = 9 #number of unique classes per label

model = build_lstm_model(neurons, input_shape, lr, num_classes, activation, loss)

early_stopings = tf.keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=2, verbose=1, mode="auto")
callbacks = [early_stopings]

trainHistrory = model.fit(
    x=x_train,
    y=[y_train[:, 0], y_train[:, 1], y_train[:, 2]],
    validation_data=(x_val, [y_val[:, 0], y_val[:, 1], y_val[:, 2]]),
    epochs= 10,
    batch_size=32,
    verbose=1,
    callbacks=callbacks,
)

#Evaluate on test data
# x_test = test_df.drop(["TARGET_0", "TARGET_1", "TARGET_2", "TARGET_3"], axis=1)
# y_test = test_df[["TARGET_0", "TARGET_1", "TARGET_2", "TARGET_3"]]

# y_test = np.array(y_test)

test_accuracy = model.evaluate([x_test], [y_test[:, 0], y_test[:, 1], y_test[:, 2]])

Epoch 1/10
2381/2381 [==============================] - 37s 14ms/step - loss: 4.1205 - output_1_loss: 1.0842 - output_2_loss: 1.3065 - output_3_loss: 1.7299 - output_1_sparse_categorical_accuracy: 0.6323 - output_2_sparse_categorical_accuracy: 0.6151 - output_3_sparse_categorical_accuracy: 0.2797 - val_loss: 4.0516 - val_output_1_loss: 1.0821 - val_output_2_loss: 1.2863 - val_output_3_loss: 1.6833 - val_output_1_sparse_categorical_accuracy: 0.6305 - val_output_2_sparse_categorical_accuracy: 0.6156 - val_output_3_sparse_categorical_accuracy: 0.3259
Epoch 2/10
2381/2381 [==============================] - 29s 12ms/step - loss: 4.0265 - output_1_loss: 1.0703 - output_2_loss: 1.2819 - output_3_loss: 1.6743 - output_1_sparse_categorical_accuracy: 0.6326 - output_2_sparse_categorical_accuracy: 0.6154 - output_3_sparse_categorical_accuracy: 0.3295 - val_loss: 4.0047 - val_output_1_loss: 1.0756 - val_output_2_loss: 1.2753 - val_output_3_loss: 1.6538 - val_output_1_sparse_categorical_accuracy: 0

In [ ]:
predicted_labels = [np.argmax(pred, axis=-1) for pred in predictions]

In [ ]:
predicted_labels[0]

array([5, 5, 1, ..., 5, 5, 5])

In [ ]:

X, y = df.drop(columns=['target1', 'target2', 'target3']), df[['target1', 'target2', 'target3']]
# Veri standartlaştırma
scaler = StandardScaler()
data_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(data_scaled,y, test_size=0.1, random_state=42)




In [ ]:
y_train

,target1,target2,target3
65710,5,1,6
82341,8,4,0
33587,8,1,4
323,5,1,2
3840,5,6,2
...,...,...,...
6265,5,1,0
54886,5,1,2
76820,5,1,6
860,8,6,2


In [ ]:
X_train

array([[ 0.94010876,  0.02149488,  0.28148241, ..., -0.11106506,
        -0.06259   , -0.45116505],
       [ 0.94010876, -0.55099891,  1.49522509, ..., -0.11106506,
        -0.06259   ,  2.21648375],
       [-0.42545704, -0.33933236,  1.58461666, ..., -0.11106506,
        -0.06259   ,  2.21648375],
       ...,
       [ 0.94010876, -0.40618411, -0.05337361, ..., -0.11106506,
        -0.06259   , -0.45116505],
       [-1.79102283,  0.98709599, -0.85689147, ..., -0.11106506,
        -0.06259   ,  2.21648375],
       [-0.42545704,  2.64279972, -0.88064255, ..., -0.11106506,
        -0.06259   , -0.45116505]])

In [ ]:
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dense

# Input layer'ları oluşturun
input1 = Input(shape=(max_sequence_length,))
input2 = Input(shape=(max_sequence_length,))
input3 = Input(shape=(max_sequence_length,))

# Embedding katmanlarını oluşturun
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_sequence_length)

# LSTM katmanlarını oluşturun
lstm_layer = LSTM(64, return_sequences=True)

# Her sütun için farklı çıkış katmanlarını oluşturun
output1 = Dense(3, activation='softmax', name='output1')(lstm_layer(embedding_layer(input1)))
output2 = Dense(3, activation='softmax', name='output2')(lstm_layer(embedding_layer(input2)))
output3 = Dense(3, activation='softmax', name='output3')(lstm_layer(embedding_layer(input3)))

# Modeli oluşturun
model = Model(inputs=[input1, input2, input3], outputs=[output1, output2, output3])

# Modeli derleyin
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitin
model.fit([X_train_padded, X_train_padded, X_train_padded], [y_train1, y_train2, y_train3], validation_data=([X_test_padded, X_test_padded, X_test_padded], [y_test1, y_test2, y_test3]), epochs=10, batch_size=64)


In [ ]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(8, activation="relu", name="layer1"),
        layers.Dense(6, activation="relu", name="layer2"),
         LayerNormalization(axis=1),
        layers.Dense(3, activation="relu", name="layer3"),
        layers.Dense(3, activation="relu", name="layer4"),
        Dropout(0.4),
        layers.Dense(6, activation="relu", name="layer5"),
        Dropout(0.4),
         LayerNormalization(axis=1),
        layers.Dense(8, activation="relu", name="layer6"),
        layers.Dense(9, name="layer7"),
    ]
)

# Modeli Derleme
model.compile(loss='categorical_crossentropy',
                optimizer=Adam(),
                metrics=['accuracy'])

history = model.fit(x=X_train,y=y_train,
                        epochs=150,
                        validation_data = (X_test,y_test),
                       callbacks=callbacks_list
                        )

Epoch 1/150
2641/2646 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.0012
Epoch 1: val_accuracy improved from -inf to 0.00096, saving model to weights-improvement-01-0.000957.hdf5
2646/2646 [==============================] - 24s 6ms/step - loss: nan - accuracy: 0.0012 - val_loss: nan - val_accuracy: 9.5694e-04
Epoch 2/150
  21/2646 [..............................] - ETA: 13s - loss: nan - accuracy: 0.0030    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2644/2646 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.0012
Epoch 2: val_accuracy did not improve from 0.00096
2646/2646 [==============================] - 15s 6ms/step - loss: nan - accuracy: 0.0012 - val_loss: nan - val_accuracy: 9.5694e-04
Epoch 3/150
2638/2646 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.0012
Epoch 3: val_accuracy did not improve from 0.00096
2646/2646 [==============================] - 15s 6ms/step - loss: nan - accuracy: 0.0012 - val_loss: nan - val_accuracy: 9.5694e-04
Epoch 4/150
2642/2646 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.0012
Epoch 4: val_accuracy did not improve from 0.00096
2646/2646 [==============================] - 15s 6ms/step - loss: nan - accuracy: 0.0012 - val_loss: nan - val_accuracy: 9.5694e-04
Epoch 5/150
2646/2646 [==============================] - ETA: 0s - loss: nan - accuracy: 0.0012
Epoch 5: val_accuracy did not improve from 0.00096
2646/2646 [=================